In [ ]:
from grid.robot.airgen_car import AirGenCar
airgen_car_0 = AirGenCar(**{'geo': False})

In [ ]:
from grid.model.perception.vo.vo_dpvo import VO
from grid.model.perception.vo.vo_dpvo import VO

import numpy as np
import airgen
import time
from airgen.utils.mechanics import to_eularian_angles
client = airgen_car_0.client

# Get camera parameters
camera_info = client.simGetCameraInfo("0")
print(f"camera into : {camera_info}")

# Initialize the VO model with calibration parameters
# match this with the camera settings while creating the session
calib = np.array([320, 320, 320, 240])
vo_model = VO(calib)

In [ ]:
from grid.model.perception.vo.vo_dpvo import VO
import rerun as rr
import numpy as np
import airgen
import time
from airgen.utils.mechanics import to_eularian_angles


# Initialize the Airgen client
# airgen_car_0 = AirGenCar(**{'geo': False})
client = airgen_car_0.client
#client.confirmConnection()
client.reset()
client.enableApiControl(False)  # User controls the vehicle


# Get the initial pose of the vehicle
initial_pose = client.simGetVehiclePose()
initial_position = initial_pose.position
initial_orientation = initial_pose.orientation
# Initialize current global pose
current_position = initial_position
current_orientation = initial_orientation

def apply_relative_pose(current_position, current_orientation, rel_pose):
    # Extract relative translation and rotation from the pose
    rel_east = rel_pose[0] 
    rel_down = -rel_pose[1] 
    rel_north = rel_pose[2] 
    rel_qx = rel_pose[3]
    rel_qy = rel_pose[4]
    rel_qz = rel_pose[5]
    rel_qw = rel_pose[6]

    new_position = airgen.Vector3r(x_val = current_position.x_val + rel_north,
    y_val = current_position.y_val + rel_east,
    z_val = current_position.z_val + rel_down)
    # Apply relative translation
    
    # get relative orientation (assuming radians)
    relative_orientation = airgen.Quaternionr(rel_qw, rel_qx , rel_qy, rel_qz)
    
    # Apply relative rotation by quaternion multiplication
    new_orientation = current_orientation * relative_orientation
    
    return new_position, new_orientation

# Main loop for image acquisition and pose computation
try:
    t = 0
    while True:
        # Capture RGB image from Airgen
        t = t + 1
        responses = client.simGetImages([airgen.ImageRequest("0", airgen.ImageType.Scene, False, False)])
        rgb_image = np.frombuffer(responses[0].image_data_uint8, dtype=np.uint8).reshape(responses[0].height, responses[0].width, 3)
        
        # Calculate pose using VO model
        relative_pose = vo_model.run(rgb_image)
        # this pose returns x,y,z,roll,pitch,yaw
        print(f"relative pose {relative_pose}")
        print('\n')
        print(relative_pose.shape)
        print('\n')
        # Apply the relative pose to the current global pose
        current_position, current_orientation = apply_relative_pose(initial_position, initial_orientation, relative_pose)

        # Print the updated global pose
        print(f"Position: x={current_position.x_val:.2f}, y={current_position.y_val:.2f}, z={current_position.z_val:.2f}")
        rr.log(f'vo/position/pos_{t}', rr.Points3D(positions=[current_position.x_val, current_position.y_val, -current_position.z_val]))

        pitch, roll, yaw = to_eularian_angles(current_orientation)
        print(f"Orientation: pitch={np.degrees(pitch):.2f}, roll={np.degrees(roll):.2f}, yaw={np.degrees(yaw):.2f}")
        
        

except KeyboardInterrupt:
    pass
     